In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("test") \
    .getOrCreate()

25/03/03 20:15:52 WARN Utils: Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.36.152.85 instead (on interface en0)
25/03/03 20:15:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/03 20:15:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
spark.version

'3.5.4'

In [3]:
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet



--2025-03-03 20:16:38--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 108.138.245.58, 108.138.245.96, 108.138.245.16, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|108.138.245.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  5.99MB/s    in 10s     

2025-03-03 20:16:48 (6.00 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [4]:
df = spark.read \
    .option("header", "true") \
    .parquet("yellow_tripdata_2024-10.parquet") \
    .repartition(4)


In [5]:
df.write.parquet("yellow_taxi/2024/10/")

In [15]:
## How many taxi trips were there on the 15th of October?
from pyspark.sql import functions as F

df = df \
    .withColumn('pickup_date', F.to_date(df.tpep_pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.tpep_dropoff_datetime))


In [16]:
df \
    .filter(df.pickup_date == "2024-10-15") \
    .count()


128893

In [24]:
# # What is the length of the longest trip in the dataset in hours?
df = df \
    .withColumn('trip_duration_hours', (F.unix_timestamp(df.tpep_dropoff_datetime) - F.unix_timestamp(df.tpep_pickup_datetime)) / 3600)

longest_trip = df.agg(F.max(F.col("trip_duration_hours")).alias("max_trip_duration")).collect()[0]["max_trip_duration"]

print(longest_trip)

162.61777777777777


In [26]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-03 20:40:40--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 108.138.245.96, 108.138.245.16, 108.138.245.58, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|108.138.245.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv.1’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-03-03 20:40:40 (29.6 MB/s) - ‘taxi_zone_lookup.csv.1’ saved [12331/12331]



In [34]:
lookup_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [29]:
lookup_zones = spark.read \
    .option("header", "true") \
    .csv("taxi_zone_lookup.csv")

lookup_zones.createOrReplaceTempView('lookup_zones')

In [38]:
joined_df = df.join(lookup_zones, df.PULocationID == lookup_zones.LocationID, how = 'inner')

joined_df \
    .groupBy('Zone') \
    .count() \
    .show()

+--------------------+------+
|                Zone| count|
+--------------------+------+
|           Homecrest|   263|
|              Corona|   496|
|    Bensonhurst West|   312|
|          Douglaston|    74|
|      Newark Airport|   555|
|Charleston/Totten...|     4|
|          Mount Hope|   339|
|      Pelham Parkway|   178|
|East Concourse/Co...|   683|
|         Marble Hill|    73|
|           Rego Park|   471|
|Upper East Side S...|191011|
|       Dyker Heights|   172|
|   Kew Gardens Hills|   245|
|       Rikers Island|     2|
|     Jackson Heights|  1760|
|             Bayside|   117|
|      Yorkville West| 69757|
|TriBeCa/Civic Center| 53604|
|          Highbridge|   426|
+--------------------+------+
only showing top 20 rows

